In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data.sampler import SubsetRandomSampler, SequentialSampler
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)


In [5]:
import os 
import glob
import pandas as pd
from torch.utils.data import Dataset
import numpy as np
import torch
from glob import glob

class TimeSeriesDataset(Dataset):
    def __init__(self, folder_nolayoff, folder_layoff, src_col):
        
        # csvs_layoffs = glob.glob(folder_layoff + "/*.csv")
        # csvs_nolayoffs = glob.glob(folder_nolayoff + "/*.csv")
        X_train = []
        Y_train = []
        all_csv_files = []
        EXT = "*.csv"  # Define the variable EXT
        for path, subdir, files in os.walk(folder_layoff):
            
            for file in glob(os.path.join(path, EXT)):
                # print(file)
                df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
                ## If length of dataset is over 90, then cut it to 90
                if len(df) > 90:
                    print("OVER LENGTH", file)
                    df = df[-90:]
                ## If length of dataset is less than 90, then skip it
                if len(df) < 90:
                    continue
                X_train.append(df[src_col].values)
                Y_train.append(float(1))
                # all_csv_files.append(file)

        for path, subdir, files in os.walk(folder_nolayoff):
            for file in glob(os.path.join(path, EXT)):
                df = pd.read_csv(file, index_col=0, parse_dates=True).sort_index()
                ## If length of dataset is over 90, then cut it to 90
                if len(df) > 90:
                    print("OVER LENGTH", file)
                    df = df[-90:]

                ## If length of dataset is less than 90, then skip it
                if len(df) < 90:
                    continue
                X_train.append(df[src_col].values)
                Y_train.append(float(0))
                # all_csv_files.append(file)





        # for directory in csvs_layoffs:
        #     for csv in os.listdir(directory):
        #         if csv.endswith(".csv"):
        #             df = pd.read_csv(csv, index_col=0, parse_dates=True).sort_index()
        #             X_train.append(df[src_col].values)
        #             Y_train.append(1)
        #     # df = pd.read_csv(csv, index_col=0, parse_dates=True).sort_index()
        #     # X_train.append(df[src_col].values)
        #     # Y_train.append(1)
        # for directory in csvs_nolayoffs:
        #     for csv in os.listdir(directory):
        #         if csv.endswith(".csv"):
        #             df = pd.read_csv(csv, index_col=0, parse_dates=True).sort_index()
        #             X_train.append(df[src_col].values)
        #             Y_train.append(0)
            
        X_train = torch.from_numpy(np.array(X_train))
        Y_train = torch.from_numpy(np.array(Y_train))
        # if len(X_train.shape) < 3:
        #     X_train = X_train.unsqueeze(2)

        # if X_train.shape.index(min(X_train.shape[1], X_train.shape[2])) != 1:  # make sure the Channels in second dim
        #     X_train = X_train.permute(0, 2, 1)

        self.X_train = X_train
        self.Y_train = Y_train
        self.num_channels = X_train.shape[1]
        self.len = X_train.shape[0]

    def __len__(self):
        return self.len
    
    def __getitem__(self, idx):
        return self.X_train[idx].float(), self.Y_train[idx]



In [25]:

dataset_opc = TimeSeriesDataset("stocks_no_layoffs", "stocks_layoffs", "open_percent_change")
dataset_oipa = TimeSeriesDataset("stocks_no_layoffs", "stocks_layoffs", "open_inproportion_to_average")
dataset_on = TimeSeriesDataset("stocks_no_layoffs", "stocks_layoffs", "open_normalized")
batch_size = 1
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(dataset_opc)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SequentialSampler(val_indices)

train_loader_opc = torch.utils.data.DataLoader(dataset_opc, batch_size=batch_size, 
                                           sampler=train_sampler)
train_loader_oipa = torch.utils.data.DataLoader(dataset_oipa, batch_size=batch_size, 
                                           sampler=train_sampler)
train_loader_on = torch.utils.data.DataLoader(dataset_on, batch_size=batch_size, 
                                           sampler=train_sampler)

validation_loader_opc = torch.utils.data.DataLoader(dataset_opc, batch_size=batch_size, sampler=valid_sampler)
validation_loader_oipa = torch.utils.data.DataLoader(dataset_oipa, batch_size=batch_size, sampler=valid_sampler)
validation_loader_on = torch.utils.data.DataLoader(dataset_on, batch_size=batch_size,   sampler=valid_sampler)

In [7]:
from torch.utils.data import DataLoader

In [8]:
train_loader_opc.dataset.Y_train

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 

In [9]:
# tdloader = TimeSeriesDataset("stocks_no_layoffs", "stocks_layoffs", "open_in_proportion_to_average")

KeyError: 'open_in_proportion_to_average'

In [ ]:
# layoff_data_loader = DataLoader(dataset=tdloader, batch_size=1, 
#                                 shuffle=True, drop_last=False, num_workers=0)

In [ ]:
# layoff_data_loader.dataset.X_train.shape

torch.Size([516, 90])

In [ ]:
# ## Iterate through dataloader and print y labels
# for i, (x, y) in enumerate(validation_loader_opc):
#     print(y)
#     # print(x)
    # break

tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], d

In [ ]:
# ## Iterate through dataloader and print y labels
# for i, (x, y) in enumerate(train_loader_oipa):
#     print(y)

tensor([1.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([0.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([1.], dtype=torch.float64)
tensor([0.], d

In [46]:
# Define the model architecture
class BinaryClassifier(nn.Module):
    def __init__(self):
        super(BinaryClassifier, self).__init__()
        self.fc1 = nn.Linear(90, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.sigmoid(self.fc3(x))
        return x

# Create an instance of the model
model_open_percent_changed = BinaryClassifier()
model_open_inproportion_to_average = BinaryClassifier()
model_open_normalized = BinaryClassifier()
# Define the loss function and optimizer
criterion_opc = nn.BCELoss()
criterion_oipa = nn.BCELoss()
criterion_on = nn.BCELoss()
optimizer_opc = optim.Adam(model_open_percent_changed.parameters())
optimizer_oipa = optim.Adam(model_open_inproportion_to_average.parameters())
optimizer_on = optim.Adam(model_open_normalized.parameters())

# Training loop
num_epochs = 50
batch_size = 32

for epoch in range(num_epochs):
    ## Run through dataloader batches
    



    for i, (X_train, y_train) in enumerate(train_loader_opc):
    
        # Forward pass
        # print(X_train)
        outputs_open_percent_changed = model_open_percent_changed(X_train)
        y_train = y_train.unsqueeze(1)
        loss_opc = criterion_opc(outputs_open_percent_changed.float(), y_train.float())
        optimizer_opc.zero_grad()
        loss_opc.backward()
        optimizer_opc.step()
    for i, (X_train, y_train) in enumerate(train_loader_oipa):
        # Forward pass
        outputs_open_inproportion_to_average = model_open_inproportion_to_average(X_train)
        y_train = y_train.unsqueeze(1)
        loss_oipa = criterion_oipa(outputs_open_inproportion_to_average.float(), y_train.float())
        optimizer_oipa.zero_grad()
        loss_oipa.backward()
        optimizer_oipa.step()
    for i, (X_train, y_train) in enumerate(train_loader_on):
        # Forward pass
        outputs_open_normalized = model_open_normalized(X_train)
        y_train = y_train.unsqueeze(1)
        loss_on = criterion_on(outputs_open_normalized.float(), y_train.float())
        # Backward pass and optimization
        optimizer_on.zero_grad()
        loss_on.backward()
        optimizer_on.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss_opc: {loss_opc.item():.4f}, Loss_oipa: {loss_oipa.item():.4f}, Loss_on: {loss_on.item():.4f}')
    

# # Evaluation
# with torch.no_grad():
#     outputs = model(X_test)
#     predicted = (outputs >= 0.5).float()
#     accuracy = (predicted == y_test).float().mean()
#     print(f'Test Accuracy: {accuracy:.4f}')

# # Make predictions
# with torch.no_grad():
#     outputs = model(X_new)
#     predicted = (outputs >= 0.5).float()

Epoch [1/50], Loss_opc: 0.6606, Loss_oipa: 0.7223, Loss_on: 0.7776
Epoch [2/50], Loss_opc: 0.6083, Loss_oipa: 0.2233, Loss_on: 1.1204
Epoch [3/50], Loss_opc: 0.6352, Loss_oipa: 0.5354, Loss_on: 0.4349
Epoch [4/50], Loss_opc: 0.5944, Loss_oipa: 0.7631, Loss_on: 0.6089
Epoch [5/50], Loss_opc: 0.6254, Loss_oipa: 0.2763, Loss_on: 0.4102
Epoch [6/50], Loss_opc: 0.4616, Loss_oipa: 0.9589, Loss_on: 0.6021
Epoch [7/50], Loss_opc: 0.2122, Loss_oipa: 0.4094, Loss_on: 0.4716
Epoch [8/50], Loss_opc: 0.0645, Loss_oipa: 0.3370, Loss_on: 0.3455
Epoch [9/50], Loss_opc: 0.1275, Loss_oipa: 0.4438, Loss_on: 0.1779
Epoch [10/50], Loss_opc: 0.6134, Loss_oipa: 0.7235, Loss_on: 0.2632
Epoch [11/50], Loss_opc: 0.2897, Loss_oipa: 0.1926, Loss_on: 0.2106
Epoch [12/50], Loss_opc: 0.3581, Loss_oipa: 0.4305, Loss_on: 1.4290
Epoch [13/50], Loss_opc: 0.0015, Loss_oipa: 0.4205, Loss_on: 0.2092
Epoch [14/50], Loss_opc: 0.3334, Loss_oipa: 0.3733, Loss_on: 0.2025
Epoch [15/50], Loss_opc: 0.3638, Loss_oipa: 1.4886, Loss_

In [ ]:
# Single model eval
# is_correct = []
# for i, (X_train, y_train) in enumerate(validation_loader_opc):
    

#         # print(X_train)
#         # print(y_train)
#         # Forward pass
#     outputs = model(X_train)
#     out = outputs.squeeze(1)
#     output_bin = (out >= 0.5).int().item()
#     correct = output_bin == int(y_train.item())
#     is_correct.append(correct)
# # Count ratio of trues and falses in is_correct
# true_count = sum(is_correct)
# false_count = len(is_correct) - true_count
# print(f"True count: {true_count}")
# print(f"False count: {false_count}")
# print(f"True ratio: {true_count / len(is_correct)}")


In [44]:
# Majority Voting Ensemble Eval
is_correct_confidence = []
is_correct_vote = []
is_correct_opc = []
is_correct_oipa = []
is_correct_on = []

for i, (X_train, y_train) in enumerate(validation_loader_opc):
        # print(X_train)
        # print(y_train)
        # Forward pass
    X_train_opc = validation_loader_opc.dataset.X_train[i].unsqueeze(0).float()
    X_train_oipa = validation_loader_oipa.dataset.X_train[i].unsqueeze(0).float()
    X_train_on = validation_loader_on.dataset.X_train[i].unsqueeze(0).float()
    Y_train_opc = validation_loader_opc.dataset.Y_train[i]
    Y_train_oipa = validation_loader_oipa.dataset.Y_train[i]
    Y_train_on = validation_loader_on.dataset.Y_train[i]
    # Sanity check
    assert (Y_train_opc.item() == Y_train_oipa.item())
    assert (Y_train_opc.item() == Y_train_on.item())

    
    # print(outputs_opc)
    outputs_opc = model_open_percent_changed(X_train_opc).squeeze(1)
    outputs_oipa = model_open_inproportion_to_average(X_train_oipa).squeeze(1)
    outputs_on = model_open_normalized(X_train_on).squeeze(1)

    ## Ensemble confidence based on average of outputs
    confidence_avg_out = (outputs_opc + outputs_oipa + outputs_on) / 3
    confidence_output_bin = (confidence_avg_out >= 0.5).int().item()

    vote_opc = (outputs_opc >= 0.5).int().item()
    is_correct_opc.append(vote_opc == int(y_train.item()))
    vote_oipa = (outputs_oipa >= 0.5).int().item()
    is_correct_oipa.append(vote_oipa == int(y_train.item()))
    vote_on = (outputs_on >= 0.5).int().item()
    is_correct_on.append(vote_on == int(y_train.item()))
    ## Ensemble voting based on majority voting
    vote = (vote_opc + vote_oipa + vote_on) >= 2
    correct_confidence = confidence_output_bin == int(y_train.item())
    correct_vote = vote == int(y_train.item())
    is_correct_confidence.append(correct_confidence)
    is_correct_vote.append(correct_vote)

    # out = outputs.squeeze(1)
    # output_bin = (out >= 0.5).int().item()
    # correct = output_bin == int(y_train.item())
    # is_correct.append(correct)
# Count ratio of trues and falses in is_correct
true_count_confidence = sum(is_correct_confidence)
false_count_confidence = len(is_correct_confidence) - true_count_confidence
print(f"True count (Confidence): {true_count_confidence}")
print(f"False count (Confidence): {false_count_confidence}")
print(f"True ratio (Confidence): {true_count_confidence / len(is_correct_confidence)}")
true_count_vote = sum(is_correct_vote)
false_count_vote = len(is_correct_vote) - true_count_vote
print(f"True count (Vote): {true_count_vote}")
print(f"False count (Vote): {false_count_vote}")
print(f"True ratio (Vote): {true_count_vote / len(is_correct_vote)}")
true_count_opc = sum(is_correct_opc)
false_count_opc = len(is_correct_opc) - true_count_opc
print(f"True count (OPC): {true_count_opc}")
print(f"False count (OPC): {false_count_opc}")
print(f"True ratio (OPC): {true_count_opc / len(is_correct_opc)}")
true_count_oipa = sum(is_correct_oipa)
false_count_oipa = len(is_correct_oipa) - true_count_oipa
print(f"True count (OIPA): {true_count_oipa}")
print(f"False count (OIPA): {false_count_oipa}")
print(f"True ratio (OIPA): {true_count_oipa / len(is_correct_oipa)}")
true_count_on = sum(is_correct_on)
false_count_on = len(is_correct_on) - true_count_on
print(f"True count (ON): {true_count_on}")
print(f"False count (ON): {false_count_on}")
print(f"True ratio (ON): {true_count_on / len(is_correct_on)}")


True count (Confidence): 49
False count (Confidence): 54
True ratio (Confidence): 0.47572815533980584
True count (Vote): 44
False count (Vote): 59
True ratio (Vote): 0.42718446601941745
True count (OPC): 9
False count (OPC): 94
True ratio (OPC): 0.08737864077669903
True count (OIPA): 47
False count (OIPA): 56
True ratio (OIPA): 0.4563106796116505
True count (ON): 71
False count (ON): 32
True ratio (ON): 0.6893203883495146


In [ ]:
# # Count ratio of trues and falses in is_correct
# true_count = sum(is_correct)
# false_count = len(is_correct) - true_count
# print(f"True count: {true_count}")
# print(f"False count: {false_count}")
# print(f"True ratio: {true_count / len(is_correct)}")

True count: 44
False count: 59
True ratio: 0.42718446601941745
